# GLAD alerts module

In [1]:
import ee
import os #à ne pas garder à priori ça ser à rien 
import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import HBox, Label
import geemap
import subprocess
import glob

ee.Initialize()

## fetch the alerts
Select the accurate Glad alerts 
- select a country from the LSIB_SIMPLE 2017 list 
- select a year 

> would be cool if I can select another AOI with a polygon selector

import them into sepal

In [11]:
#const
BUTTON_STYLE = {'button_color':'rgba(28,28,28,.99)'}
bUTTON_LAYOUT = widgets.Layout(
    width='175px',
)
BASEMAP = 'CartoDB.DarkMatter'
CENTER = [4.260202, 15.243700]

#template
country_code_field =  widgets.Text(
    value='',
    placeholder='Enter a country code',
)

year_field =  widgets.Dropdown(
    options=['2020','2019'],
    value='2019'
)

output = widgets.Output()

button = widgets.Button(
    description="Use these input!"
)
button.button_style = 'info'
button.style = BUTTON_STYLE
button.layout = bUTTON_LAYOUT

#function
def on_button_clicked(b):
    
    global country_code
    global year
    
    with output:
        button.disabled=True
        country_code_field.disabled=True
        year_field.disabled=True
        
        country_code = country_code_field.value.upper()
        year = year_field.value
        
        display_map(country_code)
        get_alerts(country_code, year)
        task = download_to_disk(country_code, year, alerts)
        #download_to_sepal()
        
def add_layer_to_map(aoi_ee):
    mapa.clear_layers()
    mapa.add_basemap(BASEMAP)
    mapa.addLayer(aoi, {'color': 'FF0000'})
    mapa.centerObject(aoi, zoom=3)
        
def display_map(country_code):
    global mapa
    global aoi
    
    mapa = geemap.Map(center=CENTER)
    mapa.clear_layers()
    mapa.clear_controls()
    mapa.add_basemap(BASEMAP)
    mapa.layout.height='200px'
    display(mapa)
    
    aoi = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co', country_code))
    add_layer_to_map(aoi)
    
def get_alerts(country_code, year):
    """ get the alerts from the GLAD project"""
    global country
    global alerts
    
    country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co', country_code))
    all_alerts  = ee.ImageCollection('projects/glad/alert/UpdResult')
    alerts = all_alerts.select('conf' + year[-2:]).mosaic().clip(country);
    
def download_to_disk(country_code, year, alerts):
    """download the tile to the GEE disk"""
    file_name = 'alerts_' + country_code + '_' + year
    task_config = {
        'image':alerts,
        'description':file_name,
        'scale': 30,
        'region':country.geometry(),
        'maxPixels': 1e10
    }
    
    task = ee.batch.Export.image.toDrive(**task_config)
    task.start()
    
    return task
    
    #creer un petit script d'évolution de l'avancé de la tache
    
def download_to_sepal():
    """download the tiles to the Sepal appropiate folder"""
    
    # Create a folder to download the glad images
    glad_dir = os.path.join(os.path.expanduser('~'), 'glad_alerts_results')
    if not os.path.exists(glad_dir):
        os.makedirs(glad_dir)
        
    #download the tif tiles in this very folder
    
    
    print("en route vers Sepal")
    
    
#run
ui1 = HBox([Label('contry code: '), country_code_field])
ui2 = HBox([Label('year: '), year_field])
button.on_click(on_button_clicked)
display(HBox([ui1, ui2, button]))
display(output)


Output()

Set the variables in SEPAL

In [12]:
# Create a folder to download the glad images
glad_dir = os.path.join(os.path.expanduser('~'), 'glad_results')+'/'
if not os.path.exists(glad_dir):
    os.makedirs(glad_dir)

alert_map   = glad_dir + "glad_2020_rdc.tif"
clump_map   = glad_dir + "tmp_clump_2020_rdc.tif"
alert_stats = glad_dir + "stats_glad_2020_rdc.txt"

In [13]:
#postprocess function 
def merge(alert_map, glad_dir):
    """ merge into a single tif files"""
    
    #create command
    command = [
        'gdal_merge.py',
        '-o', alert_map,
        '-v', '-co', '"COMPRESS=LZW"'
    ]
    
    #add the input files
    file_list = []
    for file in glob.glob(glad_dir+"alerts*.tif"):
        command.append(file)

    process = subprocess.run(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    ) 
    
    print(process.stdout)
    
def clump(alert_map, clump_map):
    """ clump the results"""
    process = subprocess.run(
        [
            'oft-clump', 
            '-i', alert_map, 
            '-o', clump_map
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    )
    
    print (process.stdout)
    
def calc(clump_map, alert_map, alert_stats):
    """Compute the statistics per each individual clump"""
    process = subprocess.run(
        [
            'oft-his', 
            '-um', clump_map, 
            '-i', alert_map, 
            '-o', alert_stats
        
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    )
    
    print(process.stdout)    

In [15]:
# merge ne marche pas avec de petit ordi, lancé au moins un m4 sinon on dépasse le buffer
merge(alert_map, glad_dir)

# work like a charm
clump(alert_map, clump_map)

# nPixel value 3 exceeds the given max input pixel value 1\nExiting and deleteting empty output

# ça ça bug

calc(clump_map, alert_map, alert_stats)


Processing file     1 of     4,  0.000% completed in 0 minutes.
Filename: /home/prambaud/glad_results/alerts_rdc_2020-0000000000-0000065536.tif
File Size: 5385x65536x1
Pixel Size: 0.000269 x -0.000269
UL:(29.863503,5.392048)   LR:(31.314732,-12.269549)
Copy 0,0,5385,65536 to 65536,0,5385,65536.

Processing file     2 of     4, 25.000% completed in 0 minutes.
Filename: /home/prambaud/glad_results/alerts_rdc_2020-0000065536-0000000000.tif
File Size: 65536x4414x1
Pixel Size: 0.000269 x -0.000269
UL:(12.201906,-12.269549)   LR:(29.863503,-13.459099)
Copy 0,0,65536,4414 to 0,65536,65536,4414.

Processing file     3 of     4, 50.000% completed in 0 minutes.
Filename: /home/prambaud/glad_results/alerts_rdc_2020-0000000000-0000000000.tif
File Size: 65536x65536x1
Pixel Size: 0.000269 x -0.000269
UL:(12.201906,5.392048)   LR:(29.863503,-12.269549)
Copy 0,0,65536,65536 to 0,0,65536,65536.

Processing file     4 of     4, 75.000% completed in 2 minutes.
Filename: /home/prambaud/glad_results/alert